In [27]:
import pandas as pd
import time
import random


from bs4 import BeautifulSoup as bs
import requests
import numpy as np

from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()

import pygame
from pygame import mixer


alimentacion = {
    "Leche Huevos Yogures y Lácteos": "https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=",
    # "Aceite Vinagre Salsas Especias" : "https://www.alcampo.es/compra-online/alimentacion/aceite-vinagre-salsas-especias/c/W18?q=%3Arelevance&page=",
    # "Desayuno y Merienda": "https://www.alcampo.es/compra-online/alimentacion/desayuno-y-merienda/c/W10?q=%3Arelevance&page=",
    # "Conservas": "https://www.alcampo.es/compra-online/alimentacion/conservas/c/W1004?q=%3Arelevance&page=",
    # "Comida Internacional":"https://www.alcampo.es/compra-online/alimentacion/comida-internacional/c/W9410?q=%3Arelevance&page=",
    # "Aperitivos, aceitunas y frutos secos": "https://www.alcampo.es/compra-online/alimentacion/aperitivos-aceitunas-y-frutos-secos/c/W120?q=%3Arelevance&page=",
    # "Nutrición deportiva":"https://www.alcampo.es/compra-online/alimentacion/nutricion-deportiva/c/W12?q=%3Arelevance&page=",
    # "Caldos, Pasta, Arroz Legumbres Puré":"https://www.alcampo.es/compra-online/alimentacion/caldos-pasta-arroz-legumbres-pure/c/W140?q=%3Arelevance&page=",
    # "Azúcar Harina Masas y Pan": "https://www.alcampo.es/compra-online/alimentacion/azucar-harina-masas-y-pan/c/W1009?q=%3Arelevance&page=",
    # "Alimentos Especiales": "https://www.alcampo.es/compra-online/alimentacion/alimentos-especiales-/c/W33?q=%3Arelevance&page=",
    # "Veganos": "https://www.alcampo.es/compra-online/alimentacion/veganos/c/W09112021?q=%3Arelevance&page=",
    # "Supermercado Ecológico": "https://www.alcampo.es/compra-online/alimentacion/supermercado-ecologico/c/W26112021?q=%3Arelevance&page="
}



dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in alimentacion.items():

    categoria = i[0]
    url_to_scrape = i[1]
    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")
     
        print(categoria,url)
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:
            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")

            for product in products:

                dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                try:
                    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))
                except:
                    print("List index out of range")
                    dict_alcampo["Price"].append("NaN")

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )
            
        contador+=1
    


df_alcampo = pd.DataFrame(dict_alcampo)

#Play sound when completed
mixer.init()
mixer.music.load(".././assets/completed.mp3")
mixer.music.set_volume(0.4)
mixer.music.play()  

df_alcampo


Leche Huevos Yogures y Lácteos https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=0
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Leche Huevos Yogures y Lácteos https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=1
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Leche Huevos Yogures y Lácteos https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=2
Category: 144 Name: 144 Image: 144 Price: 144 Stock 144
Leche Huevos Yogures y Lácteos https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=3
Category: 192 Name: 192 Image: 192 Price: 192 Stock 192
Leche Huevos Yogures y Lácteos https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=4
Category: 240 Name: 240 Image: 240 Price: 240 Stock 240
Leche Huevos Yogures y Lácteos

,Category,Name,Price,Stock,Image
0,Leche Huevos Yogures y Lácteos,Bebida vegetal de avena SANTIVERI CALCIAVENA 1 l.,"2,61 €(2,61 €/Litro)",Yes,https://www.alcampo.es/media/h34/h79/115375081...
1,Leche Huevos Yogures y Lácteos,Leche semidesnatada de vaca PRODUCTO ALCAMPO 6...,"5,40 €(0,90 €/Litro)",Yes,https://www.alcampo.es/media/h13/h14/118842518...
2,Leche Huevos Yogures y Lácteos,Leche entera de vaca PRODUCTO ALCAMPO 6 x 1 l.,"5,40 €(0,90 €/Litro)",Yes,https://www.alcampo.es/media/h0a/he2/118842514...
3,Leche Huevos Yogures y Lácteos,Leche desnatada de vaca PRODUCTO ALCAMPO 6x 1 l.,"5,34 €(0,89 €/Litro)",Yes,https://www.alcampo.es/media/h51/h3f/118842522...
4,Leche Huevos Yogures y Lácteos,Leche entera L.R. 6 x 1 l.,"5,28 €(0,88 €/Litro)",Yes,https://www.alcampo.es/media/h37/h6a/104576930...
...,...,...,...,...,...
1530,Leche Huevos Yogures y Lácteos,Yogur líquido para beber con sabor a stracciat...,"1,98 €(3,84 €/Litro)",Yes,https://www.alcampo.es/media/hc0/heb/114365156...
1531,Leche Huevos Yogures y Lácteos,Yogur griego con chcolate Valor OIKOS de Danon...,"1,68 €(7,63 €/Kg)",No,https://www.alcampo.es/media/h0a/h11/112176768...
1532,Leche Huevos Yogures y Lácteos,"Mousse de textura ligera y cremosa, con sabor ...","2,58 €(10,75 €/Kg)",No,https://www.alcampo.es/media/h32/h6a/114365505...
1533,Leche Huevos Yogures y Lácteos,Leche de cabra ecológica SATA GADEA Pack de 6 ...,"22,97 €(3,82 €/Litro)",No,https://www.alcampo.es/media/h60/hb2/117189545...


In [28]:
# Add Alcampo name
df_alcampo['Shop'] = 'Alcampo'

# Add timestamp
today = pd.Timestamp.now()
df_alcampo['Date'] = today
df_alcampo

# Append to existing csv file
df_alcampo.to_csv("../data/alcampo.csv", mode='a', index=False, header=False)

# Read in csv to save as excel as well
# read_file = pd.read_csv ("../data/alcampo.csv")
# read_file.to_excel ("../data/alcampo.xlsx", index = None, header=True)

# Read Alcampo csv
alcampo = pd.read_csv("../data/alcampo.csv")
print("Appended",today)
alcampo

Appended 2023-03-17 09:26:47.451239


,Category,Name,Price,Stock,Image,Shop,Date
0,Veganos,Bebida de avena con alto contenido de calcio y...,"5,94 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hbc/hc7/114853804...,Alcampo,2023-03-08 19:51:25.159831
1,Veganos,"Bebida de soja sin gluten, enriquecida con cal...","5,34 €(0,89 €/Litro)",Yes,https://www.alcampo.es/media/h2b/hc3/107936543...,Alcampo,2023-03-08 19:51:25.159831
2,Veganos,Bebida de avena con alto contenido de calcio y...,"0,99 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hc3/h79/109357267...,Alcampo,2023-03-08 19:51:25.159831
3,Veganos,"Bebida de almendras 100% vegetal, sin azúcar n...","2,10 €(2,10 €/Litro)",Yes,https://www.alcampo.es/media/h31/h28/114171706...,Alcampo,2023-03-08 19:51:25.159831
4,Veganos,Bebida de avena ALCAMPO ECOLÓGICO brick de 1 l...,"11,22 €(0,18 €/100ml)",Yes,https://www.alcampo.es/media/hb5/h66/111936354...,Alcampo,2023-03-08 19:51:25.159831
...,...,...,...,...,...,...,...
163713,Leche Huevos Yogures y Lácteos,Yogur líquido para beber con sabor a stracciat...,"1,98 €(3,84 €/Litro)",Yes,https://www.alcampo.es/media/hc0/heb/114365156...,Alcampo,2023-03-17 09:26:47.451239
163714,Leche Huevos Yogures y Lácteos,Yogur griego con chcolate Valor OIKOS de Danon...,"1,68 €(7,63 €/Kg)",No,https://www.alcampo.es/media/h0a/h11/112176768...,Alcampo,2023-03-17 09:26:47.451239
163715,Leche Huevos Yogures y Lácteos,"Mousse de textura ligera y cremosa, con sabor ...","2,58 €(10,75 €/Kg)",No,https://www.alcampo.es/media/h32/h6a/114365505...,Alcampo,2023-03-17 09:26:47.451239
163716,Leche Huevos Yogures y Lácteos,Leche de cabra ecológica SATA GADEA Pack de 6 ...,"22,97 €(3,82 €/Litro)",No,https://www.alcampo.es/media/h60/hb2/117189545...,Alcampo,2023-03-17 09:26:47.451239


In [10]:
import pygame
from pygame import mixer

def playSound():
    mixer.init()
    mixer.music.load(".././assets/completed.mp3")
    mixer.music.set_volume(0.4)
    mixer.music.play()

playSound()